In [1]:
import requests 
import pandas as pd
import numpy as np
import time
import logging
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup

In [3]:
logging.basicConfig(filename='lol_data.log', 
                    encoding='utf-8', 
                    level=logging.DEBUG)

In [6]:
#function that gets the html of all solo queue matches from league of Graphs of a given summoner
def lographs_htmlScrapper(summoner):
        driver = webdriver.Edge('C:\Git Projects\edgedriver_win64\msedgedriver.exe')
        driver.get("https://www.leagueofgraphs.com/summoner/lan/" + summoner)
        
        menu = driver.find_element_by_css_selector('a[data-dropdown="drop-queueTypes-recentgames"]')
        menu_item = driver.find_element_by_css_selector('a[data-additional-queue-id="420"]')
        ActionChains(driver).click(menu).pause(1).click(menu_item).pause(1)\
                            .send_keys(Keys.END).pause(np.random.randint(4)).perform()        #select option from menu
                    #while button exists get more matches 
        button_css = 'button[class="see_more_ajax_button"]'
        logging.debug("Getting data from the page..")
        while(True): 
            try:            
                button = driver.find_element_by_css_selector(button_css)
                ActionChains(driver).click(button).pause(np.random.randint(4)).send_keys(Keys.END).pause(1).perform()
            except:
                print("No more matches")
                break
                
        page_source = driver.page_source
        driver.close()
        return page_source

In [7]:
#function that takes a list of summoner names and creates a list of dataframes for each summoner in the lis
def create_dataframe(summoners):
    match_history = []
    logging.debug("Creating a dataframe for each summoner...")
    for summoner in summoners:
            matches = pd.read_html(lographs_htmlScrapper(summoner))[0] #there are many tables in the page, we only want the first
            matches = matches.iloc[:,[2,4]] #take the columns we need
            matches.columns = ['Result','Score']
            match_history.append(matches)
            
    return match_history

In [8]:
#As re.findall returns a list we need to get the data of each list, this functions does that
def data_from_list(lst):
    temp = []
    logging.debug("Getting the data from the lists...")
    for a in lst:
        for b in a:
            temp.append(b)
    return temp

In [9]:
#the function create_dataframe only create a df without separating the info, we do that here
def df_cleaning(df):
    results = [item.lstrip() for item in df['Result']]
    score = [item.lstrip() for item in df['Score']]
    status = []
    date = []
    duration= []
    kda =[]
    cs = []
    
    logging.debug("Getting the data from the dataframes and creating columns for each data...")
    for result,score in zip(results,score):
        status.append(re.findall(r'^(Defeat|Victory|Remake)',result))
        date.append(re.findall(r'\d+ days ago',result))
        duration.append(re.findall(r'\d+min',result))                
        kda.append(re.findall(r'\d+\s+/\s+\d+\s+/\s+\d+',score))
        cs.append(re.findall(r'\d+ CS',score))

    status = data_from_list(status)
    date = data_from_list(date)
    duration= data_from_list(duration)
    kda =data_from_list(kda)
    cs = data_from_list(cs)
                              
    new_df = pd.DataFrame(list(zip(status,date,duration,kda,cs)))
    new_df.columns = ["W/L/R", "Date" , "Duration" ,"KDA" , "CS"]
    return new_df

In [10]:
#function that takes a list of dataframes, and a list of names and saves each data frame with the names
def save_dataframes(dataframes, names):
    for dataframe, name in zip(dataframes, names):
        if ' ' in name:
            name = re.sub(' ','_',name)           
        dataframe.to_csv(f'match_histories/{name}_match_history.csv')

# And it all comes down to this

In [11]:
#Creating a list of dataframes from different summoners
summoners = ["Lopso","Cosa","WeIinton Quiw", "Rudon", "El Brayayin", "Twitch Bejjaniii", "Eipa", "Don Miguel", "Aggression", "DiaCraOne", "IvanRomeroEnFB"]
match_history = create_dataframe(summoners)

#now we just create a list of 'clean' dataframes
clean_match_history = []
for match in match_history:
    clean_match_history.append(df_cleaning(match))

#and we save those dataframes
try:   
    save_dataframes(clean_match_history, summoners)
    logging.info('Dataframes saved!')
except:
    logging.exception('An Exception ocurred:')

No more matches
No more matches
No more matches
No more matches
No more matches
No more matches
No more matches
No more matches
No more matches
No more matches
No more matches
